In [1]:
#Mediante https://www.tensorflow.org/tutorials/structured_data/time_series
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from tensorflow.keras import layers
from keras.layers import Dense,LSTM
from keras.models import Sequential, Model
from keras import backend
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

Using TensorFlow backend.


In [2]:
#Recuperación del Dataset generado anteriormente
#dataMeteoCellId = pd.read_csv('transformed_data/tres_semanas_mi_tweet_meteo.csv', index_col=0)
#dataMeteoCellId.head(10)

In [ ]:
dataMeteoCellId5459 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_5459.csv', index_col=0)
dataMeteoCellId5459

In [ ]:
dataMeteoCellId5570 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_5570.csv', index_col=0)
dataMeteoCellId5570

In [ ]:
dataMeteoCellId5644 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_5644.csv', index_col=0)
dataMeteoCellId5644

In [ ]:
dataMeteoCellId6372 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_6372.csv', index_col=0)
dataMeteoCellId6372

In [ ]:
dataMeteoCellId6377 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_6377.csv', index_col=0)
dataMeteoCellId6377

In [ ]:
dataMeteoCellId6561 = pd.read_csv('transformed_data/mi_CellId_extra_info_10min_6561.csv', index_col=0)
dataMeteoCellId6561

In [6]:
TRAIN_SPLIT = math.ceil(dataMeteoCellIdMatrix[0][1].shape[0] * 0.7)

In [7]:
print (TRAIN_SPLIT)

453


In [8]:
tf.random.set_seed(13)

In [9]:
#features_considered = ['weekday', 'hour', 'temperature', 'internet', 'relativeHumidity']
features_considered = ['internet']
index_output = 0

In [10]:
dataset = []
for MeteoCellId in dataMeteoCellIdMatrix:
    uniDataAux = MeteoCellId[1][features_considered]
    uniDataAux.index = MeteoCellId[1]['hour'] + MeteoCellId[1]['weekday'] * 24 + (MeteoCellId[1]['week'] - 48) * 24 * 7
    dataset.append(uniDataAux.values)

In [11]:
dataset

[array([[2212.23393517],
        [1694.4795715 ],
        [1581.29573798],
        [1404.36771051],
        [1399.00782609],
        [1570.05617291],
        [2435.48514499],
        [4288.21099912],
        [5976.3072072 ],
        [6375.77623316],
        [6893.95921089],
        [7044.52139339],
        [8150.78252469],
        [7778.52922952],
        [7021.77022965],
        [7098.26933221],
        [7217.22894651],
        [6134.50626999],
        [5431.54778574],
        [4451.15232191],
        [4379.74322466],
        [4311.34950049],
        [4600.10805545],
        [3064.07668443],
        [2192.29074564],
        [1777.40998735],
        [1593.27372145],
        [1409.82198597],
        [1276.1606656 ],
        [1334.66503215],
        [1693.43237964],
        [2190.75800838],
        [2808.08161924],
        [3451.61160103],
        [4417.88093897],
        [4502.78121908],
        [5223.90953802],
        [5329.36537474],
        [5415.86334951],
        [6059.38238616],


In [12]:
len(dataset)

54

In [13]:
dataset[0].shape[0]

647

In [14]:
dataset[0][1].shape[0] * len(dataset)

54

In [ ]:
uni_data5459 = dataMeteoCellId5459[features_considered]
uni_data5459.index = dataMeteoCellId5459['hour'] + dataMeteoCellId5459['weekday'] * 24 + (dataMeteoCellId5459['week'] - 48) * 24 * 7
uni_data5459.head()

In [ ]:
dataset5459 = uni_data5459.values;

In [ ]:
uni_data5570 = dataMeteoCellId5570[features_considered]
uni_data5570.index = dataMeteoCellId5570['hour'] + dataMeteoCellId5570['weekday'] * 24 + (dataMeteoCellId5570['week'] - 48) * 24 * 7
uni_data5570.head()

In [ ]:
dataset5570 = uni_data5570.values;

In [ ]:
uni_data5644 = dataMeteoCellId5644[features_considered]
uni_data5644.index = dataMeteoCellId5644['hour'] + dataMeteoCellId5644['weekday'] * 24 + (dataMeteoCellId5644['week'] - 48) * 24 * 7
uni_data5644.head()

In [ ]:
dataset5644 = uni_data5644.values;

In [ ]:
uni_data6372 = dataMeteoCellId6372[features_considered]
uni_data6372.index = dataMeteoCellId6372['hour'] + dataMeteoCellId6372['weekday'] * 24 + (dataMeteoCellId6372['week'] - 48) * 24 * 7
uni_data6372.head()

In [ ]:
dataset6372 = uni_data6372.values;

In [ ]:
uni_data6377 = dataMeteoCellId6377[features_considered]
uni_data6377.index = dataMeteoCellId6377['hour'] + dataMeteoCellId6377['weekday'] * 24 + (dataMeteoCellId6377['week'] - 48) * 24 * 7
uni_data6377.head()

In [ ]:
dataset6377 = uni_data6377.values;

In [ ]:
uni_data6561 = dataMeteoCellId6561[features_considered]
uni_data6561.index = dataMeteoCellId6561['hour'] + dataMeteoCellId6561['weekday'] * 24 + (dataMeteoCellId6561['week'] - 48) * 24 * 7
uni_data6561.head()

In [ ]:
dataset6561 = uni_data6561.values;

In [15]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [16]:
univariate_past_history = 3
univariate_future_target = 0

In [17]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
              labels.append(target[i+target_size])
        else:
              labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In [18]:
past_history = 3
future_target = 0
STEP = 1

BATCH_SIZE = 16
BUFFER_SIZE = 1

In [19]:
train_multi = []
val_multi = []
for data in dataset:
    train_multi.append(multivariate_data(data, data[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True))
    val_multi.append(multivariate_data(data, data[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True))

In [20]:
for data_train in train_multi:
    try:
        x_train_multi = np.concatenate((x_train_multi, data_train[0]), axis=0)
    except NameError:
        print (data_train[0])
        x_train_multi = np.array(data_train[0])

[[[2212.23393517]
  [1694.4795715 ]
  [1581.29573798]]

 [[1694.4795715 ]
  [1581.29573798]
  [1404.36771051]]

 [[1581.29573798]
  [1404.36771051]
  [1399.00782609]]

 ...

 [[2426.24172745]
  [1837.52333856]
  [1759.9699805 ]]

 [[1837.52333856]
  [1759.9699805 ]
  [1397.88317199]]

 [[1759.9699805 ]
  [1397.88317199]
  [1193.5088533 ]]]


In [21]:
for data_train in train_multi:
    print(data_train[1])
    try:
        y_train_multi = np.concatenate((y_train_multi, data_train[1]), axis=0)
    except NameError:
        y_train_multi = np.array(data_train[1])

[1404.36771051 1399.00782609 1570.05617291 2435.48514499 4288.21099912
 5976.3072072  6375.77623316 6893.95921089 7044.52139339 8150.78252469
 7778.52922952 7021.77022965 7098.26933221 7217.22894651 6134.50626999
 5431.54778574 4451.15232191 4379.74322466 4311.34950049 4600.10805545
 3064.07668443 2192.29074564 1777.40998735 1593.27372145 1409.82198597
 1276.1606656  1334.66503215 1693.43237964 2190.75800838 2808.08161924
 3451.61160103 4417.88093897 4502.78121908 5223.90953802 5329.36537474
 5415.86334951 6059.38238616 6137.51510956 5371.0963032  5504.45942505
 4518.27322306 5232.65292583 4239.76883279 4058.59039484 3043.58449326
 2942.21302868 2183.92872034 1808.99178303 1391.92247749 1306.89493829
 1277.16210659 1517.58830113 1632.02801669 1965.61386574 2458.73441667
 3777.13122797 3315.65455835 3599.33145137 5089.10404963 5005.77066401
 5383.71529779 5133.2193214  4665.70361583 4447.93888417 3635.51844383
 3342.2423345  2679.47746547 1716.61461953 1618.46368839 1348.55060032
 1194.

In [22]:
for data_val in val_multi:
    try:
        x_val_multi = np.concatenate((x_val_multi, data_val[0]), axis=0)
    except NameError:
        x_val_multi = np.array(data_val[0])

In [23]:
for data_val in val_multi:
    try:
        y_val_multi = np.concatenate((y_val_multi, data_val[1]), axis=0)
    except NameError:
        y_val_multi = np.array(data_val[1])

In [ ]:
dataset5459[:, index_output]

In [ ]:
x_train_multi5459, y_train_multi5459 = multivariate_data(dataset5459, dataset5459[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi5459, y_val_multi5459 = multivariate_data(dataset5459, dataset5459[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
print(x_train_multi5459.shape, y_train_multi5459.shape, x_val_multi5459.shape, y_val_multi5459.shape)

In [ ]:
x_train_multi5459.size

In [ ]:
x_train_multi5459

In [ ]:
y_train_multi5459.size

In [ ]:
x_train_multi5570, y_train_multi5570 = multivariate_data(dataset5570, dataset5570[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi5570, y_val_multi5570 = multivariate_data(dataset5570, dataset5570[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
x_train_multi5644, y_train_multi5644 = multivariate_data(dataset5644, dataset5644[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi5644, y_val_multi5644 = multivariate_data(dataset5644, dataset5644[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
x_train_multi6372, y_train_multi6372 = multivariate_data(dataset6372, dataset6372[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi6372, y_val_multi6372 = multivariate_data(dataset6372, dataset6372[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
x_train_multi6377, y_train_multi6377 = multivariate_data(dataset6377, dataset6377[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi6377, y_val_multi6377 = multivariate_data(dataset6377, dataset6377[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
x_train_multi6561, y_train_multi6561 = multivariate_data(dataset6561, dataset6561[:, index_output], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_multi6561, y_val_multi6561 = multivariate_data(dataset6561, dataset6561[:, index_output],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
x_train_multi = np.concatenate((x_train_multi5459, x_train_multi5570, x_train_multi5644, x_train_multi6372, x_train_multi6377, x_train_multi6561), axis=0)
y_train_multi = np.concatenate((y_train_multi5459, y_train_multi5570, y_train_multi5644, y_train_multi6372, y_train_multi6377, y_train_multi6561), axis=0)

x_val_multi = np.concatenate((x_val_multi5459, x_val_multi5570, x_val_multi5644, x_val_multi6372, x_val_multi6377, x_val_multi6561), axis=0)
y_val_multi = np.concatenate((y_val_multi5459, y_val_multi5570, y_val_multi5644, y_val_multi6372, y_val_multi6377, y_val_multi6561), axis=0)

In [24]:
x_train_multi.size

72900

In [25]:
y_train_multi.size

24300

In [26]:
x_val_multi.size

33885

In [27]:
y_val_multi.size

11295

In [ ]:
#train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
#train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
#val_data_multi = val_data_multi.batch(BATCH_SIZE)

In [ ]:
#train_data_multi5459 = tf.data.Dataset.from_tensor_slices((x_train_multi5459, y_train_multi5459))
#train_data_multi5459 = train_data_multi5459.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi5459 = tf.data.Dataset.from_tensor_slices((x_val_multi5459, y_val_multi5459))
#val_data_multi5459 = val_data_multi5459.batch(BATCH_SIZE).repeat()

In [ ]:
#train_data_multi5570 = tf.data.Dataset.from_tensor_slices((x_train_multi5570, y_train_multi5570))
#train_data_multi5570 = train_data_multi5570.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi5570 = tf.data.Dataset.from_tensor_slices((x_val_multi5570, y_val_multi5570))
#val_data_multi5570 = val_data_multi5459.batch(BATCH_SIZE).repeat()

In [ ]:
#train_data_multi5644 = tf.data.Dataset.from_tensor_slices((x_train_multi5644, y_train_multi5644))
#train_data_multi5644 = train_data_multi5644.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi5644 = tf.data.Dataset.from_tensor_slices((x_val_multi5644, y_val_multi5644))
#val_data_multi5644 = val_data_multi5644.batch(BATCH_SIZE).repeat()

In [ ]:
#train_data_multi6372 = tf.data.Dataset.from_tensor_slices((x_train_multi6372, y_train_multi6372))
#train_data_multi6372 = train_data_multi6372.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi6372 = tf.data.Dataset.from_tensor_slices((x_val_multi6372, y_val_multi6372))
#val_data_multi6372 = val_data_multi6372.batch(BATCH_SIZE).repeat()

In [ ]:
#train_data_multi6377 = tf.data.Dataset.from_tensor_slices((x_train_multi6377, y_train_multi6377))
#train_data_multi6377 = train_data_multi6377.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi6377 = tf.data.Dataset.from_tensor_slices((x_val_multi6377, y_val_multi6377))
#val_data_multi6377 = val_data_multi6377.batch(BATCH_SIZE).repeat()

In [ ]:
#train_data_multi6561 = tf.data.Dataset.from_tensor_slices((x_train_multi6561, y_train_multi6561))
#train_data_multi6561 = train_data_multi6561.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

#val_data_multi6561 = tf.data.Dataset.from_tensor_slices((x_val_multi6561, y_val_multi6561))
#val_data_multi6561 = val_data_multi6561.batch(BATCH_SIZE).repeat()

In [ ]:
#Codigo seguramente erroneo
#train_data_multi = train_data_multi5459.concatenate(train_data_multi5570).concatenate(train_data_multi5644).concatenate(train_data_multi6372).concatenate(train_data_multi6377).concatenate(train_data_multi6561)
#val_data_multi = train_data_multi5459.concatenate(val_data_multi5570).concatenate(val_data_multi5644).concatenate(val_data_multi6372).concatenate(val_data_multi6377).concatenate(val_data_multi6561)

In [ ]:
#list(train_data_multi.as_numpy_iterator())

In [28]:
x_train_multi.shape[-2:]

(3, 1)

In [29]:
train_data_multi

NameError: name 'train_data_multi' is not defined

In [47]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(256,
                                           input_shape=x_train_multi.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

single_step_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

In [48]:
EVALUATION_INTERVAL = 1
EPOCHS = 100

In [49]:
single_step_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


In [44]:
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                    patience=2,
#                                                    mode='min')

#single_step_history = single_step_model.fit(train_data_multi, epochs=50,
#                                            steps_per_epoch=EVALUATION_INTERVAL,
#                                            validation_data=val_data_multi,
#                                            callbacks=[early_stopping])

In [50]:
single_step_history = single_step_model.fit(x_train_multi, y_train_multi, epochs=EPOCHS,
                                            batch_size=1,
#                                            validation_data=(x_val_multi, y_val_multi),
                                            verbose = 1)

Epoch 1/100
24300/24300 [==============================] - 82s 3ms/step - loss: 5190824.8971 - accuracy: 0.0000e+00
Epoch 2/100
24300/24300 [==============================] - 82s 3ms/step - loss: 5252030.4572 - accuracy: 0.0000e+00
Epoch 3/100
24300/24300 [==============================] - 79s 3ms/step - loss: 5203251.6165 - accuracy: 0.0000e+00
Epoch 4/100
24300/24300 [==============================] - 78s 3ms/step - loss: 5422402.2252 - accuracy: 0.0000e+00
Epoch 5/100
24300/24300 [==============================] - 78s 3ms/step - loss: 5247404.3028 - accuracy: 0.0000e+00
Epoch 6/100
24300/24300 [==============================] - 82s 3ms/step - loss: 5208427.1871 - accuracy: 0.0000e+00
Epoch 7/100
24300/24300 [==============================] - 82s 3ms/step - loss: 5306484.7082 - accuracy: 0.0000e+00
Epoch 8/100
24300/24300 [==============================] - 80s 3ms/step - loss: 5198254.8306 - accuracy: 0.0000e+00
Epoch 9/100
24300/24300 [==============================] - 77s 3ms/step 

In [ ]:
print(single_step_model.predict(x_val_multi).shape)

In [ ]:
# plot history
plt.plot(single_step_history.history['loss'], label='train')
plt.plot(single_step_history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
score = single_step_model.evaluate(TrainX, TrainY, verbose=0)
print('Keras Model Loss = ',score[0])
print('Keras Model Accuracy = ',score[1])

In [ ]:
def plot_train_history_mse(history, title):
    mse = history.history['mse']
    val_mse = history.history['val_mse']

    epochs = range(len(mse))

    plt.figure()

    plt.plot(epochs, mse, 'b', label='Training loss')
    plt.plot(epochs, val_mse, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('MSE')

    plt.show()

In [ ]:
plot_train_history_mse(single_step_history,
                   'Single Step Training and validation loss')

In [ ]:
train_data_multi

In [ ]:
x_train_multi.size

In [ ]:
y_train_multi = y_train_multi5459 + y_train_multi5570 + y_train_multi5644 + y_train_multi6372 + y_train_multi6377 + y_train_multi6561

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 

uni_data5459.plot(kind='line',x='index',y='internet', label = "CellId: 5459")
uni_data5570.plot(kind='line',x='index',y='internet', color='red', label = "CellId: 5570")
uni_data5644.plot(kind='line',x='index',y='internet', color='green', label = "CellId: 5644")
uni_data6372.plot(kind='line',x='index',y='internet', color='orange', label = "CellId: 6372")
uni_data6377.plot(kind='line',x='index',y='internet', color='black', label = "CellId: 6377")
uni_data6561.plot(kind='line',x='index',y='internet', color='pink', label = "CellId: 6561")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('Prueba.png',bbox_inches='tight')

plt.show()

In [ ]:
uni_data = pd.concat([uni_data5459, uni_data5570, uni_data5644, uni_data6372, uni_data6377, uni_data6561 ], axis=1, keys=['5459', '5570', '5644', '6372', '6377', '6561'])

In [ ]:
uni_data.plot()

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 
uni_data.plot(subplots=True)
plt.show()

In [ ]:
uni_data

In [ ]:
uni_data = uni_data.values

In [ ]:
uni_data

In [ ]:
uni_data
len(uni_data)

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

In [ ]:
uni_train_mean

In [ ]:
uni_train_std

In [ ]:
uni_data = (uni_data-uni_train_mean)/uni_train_std

In [ ]:
uni_data

In [ ]:
uni_data[:,0]

In [ ]:
len(x_train_uni)

In [ ]:
len (x_val_uni)

In [ ]:
x_train_uni_cellId = []
y_train_uni_cellId = []
x_val_uni_cellId = []
y_val_uni_cellId = []
for i in range(0, uni_data.shape[1]):
    x_train_uni_frame, y_train_uni_frame = univariate_data(uni_data[:,i], 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
    print (len(x_train_uni_frame))
    print (type (x_train_uni_frame))
    x_val_uni_frame, y_val_uni_frame = univariate_data(uni_data[:,i], TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)
    x_train_uni_cellId.append(x_train_uni_frame)
    y_train_uni_cellId.append(y_train_uni_frame)
    x_val_uni_cellId.append(x_val_uni_frame)
    y_val_uni_cellId.append(y_val_uni_frame)
    if i == 0:
        x_train_uni = x_train_uni_frame
        y_train_uni = y_train_uni_frame
        x_val_uni = x_val_uni_frame
        y_val_uni = y_val_uni_frame
    else: 
        x_train_uni = np.concatenate((x_train_uni, x_train_uni_frame))
        y_train_uni = np.concatenate((y_train_uni, y_train_uni_frame))
        x_val_uni = np.concatenate((x_val_uni, x_val_uni_frame))
        y_val_uni = np.concatenate((y_val_uni, y_val_uni_frame))

In [ ]:
len(y_train_uni)

In [ ]:
len(x_train_uni_cellId)

In [ ]:
x_train_uni

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target Internet charge to predict')
print (y_train_uni[0])

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10,
                label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+3)*2])
    plt.xlabel('Time-Step')
    return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

In [ ]:
show_plot([x_train_uni[24], y_train_uni[24]], 0, '2nd Sample Example')

In [ ]:
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# Baseline

In [ ]:
def baseline(history):
    return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

# Common constants

In [ ]:
VAL_OPTIMIZER = 'adam'
VAL_LOSS = 'mse'
VAL_METRICS = ['accuracy', 'mae', 'mse',tf.keras.metrics.RootMeanSquaredError()]
EPOCHS = 100
VAL_BATCH_SIZE = 16

# Recurrent neural network con la CellId 5459

In [ ]:
print (x_train_uni_cellId[0].shape[-2:])

In [ ]:
simple_lstm_model_5459 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459.summary()

simple_lstm_model_5459.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
history_5459 = simple_lstm_model_5459.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=VAL_BATCH_SIZE,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[0][i*5], y_val_uni_cellId[0][i*5],
                    simple_lstm_model_5459.predict(x_val_uni_cellId[0])[i*5]], 0, 'Simple LSTM model CellId 5459')
    plot.show()

In [ ]:
test_predictions_5459 = simple_lstm_model_5459.predict(x_val_uni_cellId[0]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[0], test_predictions_5459)
plt.title('Prediction accuracy CellId 5459')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_5459 - y_val_uni_cellId[0]
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
test_predictions_5459_batch_16 = simple_lstm_model_5459.predict(x_val_uni_cellId[0])

In [ ]:
#MSE
mse_5459_batch_16 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_16(y_val_uni_cellId[0], test_predictions_5459_batch_16).numpy()

In [ ]:
#RMSE
rmse_5459_batch_16 = tf.keras.metrics.RootMeanSquaredError()
rmse_5459_batch_16(y_val_uni_cellId[0], test_predictions_5459_batch_16).numpy()

In [ ]:
#RMSE
rmse_5459_batch_16 = tf.keras.metrics.RootMeanSquaredError()
rmse_5459_batch_16.update_state(y_val_uni_cellId[0], test_predictions_5459_batch_16)
rmse_5459_batch_16.result().numpy()                                

## Cambio valores CellId 5459

#### Batch 8

In [ ]:
VAL_BATCH_SIZE_8 = 8
simple_lstm_model_5459_batch_8 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_8.summary()

simple_lstm_model_5459_batch_8.compile(optimizer=VAL_OPTIMIZER , loss=VAL_LOSS ,metrics=VAL_METRICS)

In [ ]:
history_5459_batch_8 = simple_lstm_model_5459_batch_8.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=VAL_BATCH_SIZE_8,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
history_5459_batch_8.history

In [ ]:
test_predictions_5459_batch_8 = simple_lstm_model_5459_batch_8.predict(x_val_uni_cellId[0])

mse_5459_batch_8 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_8(y_val_uni_cellId[0], test_predictions_5459_batch_8).numpy()

In [ ]:
#RMSE
mse_5459_batch_8 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_8(y_val_uni_cellId[0], test_predictions_5459_batch_8).numpy()

In [ ]:
history_dict_5459_batch_8 = history_5459_batch_8.history
mse_values = history_dict_5459_batch_8['mse']
val_mse_values_5459_batch_8 = history_dict_5459_batch_8['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_8) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_8.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_8, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 8')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 32

In [ ]:
simple_lstm_model_5459_batch_32 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_32.summary()

simple_lstm_model_5459_batch_32.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
history_5459_batch_32 = simple_lstm_model_5459_batch_32.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=32,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_32 = simple_lstm_model_5459_batch_32.predict(x_val_uni_cellId[0])

mse_5459_batch_32 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_32(y_val_uni_cellId[0], test_predictions_5459_batch_32).numpy()

In [ ]:
#RMSE
mse_5459_batch_32 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_32(y_val_uni_cellId[0], test_predictions_5459_batch_32).numpy()

In [ ]:
history_dict_5459_batch_32 = history_5459_batch_32.history
mse_values = history_dict_5459_batch_32['mse']
val_mse_values_5459_batch_32 = history_dict_5459_batch_32['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_32) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_32.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_32, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 32')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 64

In [ ]:
simple_lstm_model_5459_batch_64 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_64.summary()

simple_lstm_model_5459_batch_64.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS ,metrics=VAL_METRICS )

In [ ]:
history_5459_batch_64 = simple_lstm_model_5459_batch_64.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=64,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_64 = simple_lstm_model_5459_batch_64.predict(x_val_uni_cellId[0])

mse_5459_batch_64 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_64(y_val_uni_cellId[0], test_predictions_5459_batch_64).numpy()

In [ ]:
#RMSE
mse_5459_batch_64 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_64(y_val_uni_cellId[0], test_predictions_5459_batch_64).numpy()

In [ ]:
history_dict_5459_batch_64 = history_5459_batch_64.history
mse_values = history_dict_5459_batch_64['mse']
val_mse_values_5459_batch_64 = history_dict_5459_batch_64['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_64) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_64.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_64, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 64')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 128

In [ ]:
simple_lstm_model_5459_batch_128 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_128.summary()

simple_lstm_model_5459_batch_128.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS ,metrics=VAL_METRICS )

In [ ]:
history_5459_batch_128 = simple_lstm_model_5459_batch_128.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=128,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_128 = simple_lstm_model_5459_batch_128.predict(x_val_uni_cellId[0])

mse_5459_batch_128 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_128(y_val_uni_cellId[0], test_predictions_5459_batch_128).numpy()

In [ ]:
#RMSE
mse_5459_batch_128 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_128(y_val_uni_cellId[0], test_predictions_5459_batch_128).numpy()

In [ ]:
history_dict_5459_batch_128 = history_5459_batch_128.history
mse_values = history_dict_5459_batch_128['mse']
val_mse_values_5459_batch_128 = history_dict_5459_batch_128['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_128) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_128.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_128, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 128')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 256

In [ ]:
simple_lstm_model_5459_batch_256 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_256.summary()

simple_lstm_model_5459_batch_256.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS ,metrics=VAL_METRICS )

In [ ]:
history_5459_batch_256 = simple_lstm_model_5459_batch_256.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=256,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_256 = simple_lstm_model_5459_batch_256.predict(x_val_uni_cellId[0])

mse_5459_batch_256 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_256(y_val_uni_cellId[0], test_predictions_5459_batch_256).numpy()

In [ ]:
#RMSE
mse_5459_batch_256 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_256(y_val_uni_cellId[0], test_predictions_5459_batch_256).numpy()

In [ ]:
history_dict_5459_batch_256 = history_5459_batch_256.history
mse_values = history_dict_5459_batch_256['mse']
val_mse_values_5459_batch_256 = history_dict_5459_batch_256['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_256) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_256.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_256, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 256')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 512

In [ ]:
simple_lstm_model_5459_batch_512 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_512.summary()

simple_lstm_model_5459_batch_512.compile(optimizer=VAL_OPTIMIZER , loss=VAL_LOSS ,metrics=VAL_METRICS )

In [ ]:
EPOCHS = 100

history_5459_batch_512 = simple_lstm_model_5459_batch_512.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=512,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_512 = simple_lstm_model_5459_batch_512.predict(x_val_uni_cellId[0])

mse_5459_batch_512 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_512(y_val_uni_cellId[0], test_predictions_5459_batch_512).numpy()

In [ ]:
#RMSE
mse_5459_batch_512 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_512(y_val_uni_cellId[0], test_predictions_5459_batch_512).numpy()

In [ ]:
history_dict_5459_batch_512 = history_5459_batch_512.history
mse_values = history_dict_5459_batch_512['mse']
val_mse_values_5459_batch_512 = history_dict_5459_batch_512['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_512) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_512.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_512, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 512')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch 1024

In [ ]:
simple_lstm_model_5459_batch_1024 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[0].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5459_batch_1024.summary()

simple_lstm_model_5459_batch_1024.compile(optimizer=VAL_OPTIMIZER , loss=VAL_LOSS ,metrics=VAL_METRICS )

In [ ]:
EPOCHS = 100

history_5459_batch_1024 = simple_lstm_model_5459_batch_1024.fit(x_train_uni_cellId[0], y_train_uni_cellId[0], epochs=EPOCHS,
                      batch_size=1024,
                      validation_data=(x_val_uni_cellId[0], y_val_uni_cellId[0]), verbose=1)

In [ ]:
test_predictions_5459_batch_1024 = simple_lstm_model_5459_batch_1024.predict(x_val_uni_cellId[0])

mse_5459_batch_1024 = tf.keras.losses.MeanSquaredError()
mse_5459_batch_1024(y_val_uni_cellId[0], test_predictions_5459_batch_1024).numpy()

In [ ]:
#RMSE
mse_5459_batch_1024 = tf.keras.metrics.RootMeanSquaredError()
mse_5459_batch_1024(y_val_uni_cellId[0], test_predictions_5459_batch_1024).numpy()

In [ ]:
history_dict_5459_batch_1024 = history_5459_batch_1024.history
mse_values = history_dict_5459_batch_1024['mse']
val_mse_values_5459_batch_1024 = history_dict_5459_batch_1024['val_mse']
epochs = range(1, len(val_mse_values_5459_batch_1024) + 1)

%matplotlib inline 

plt.figure(figsize=(16, 12), dpi=100) 
plt.plot(epochs, history_5459_batch_1024.history['mse'], 'rx', label='Training data')
plt.plot(epochs, val_mse_values_5459_batch_1024, 'b', label='Test data')
plt.title('Training and test MSE CellId 5459 Batch 1024')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

#### Batch Comparation

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 

plt.plot(epochs, history_dict_5459_batch_8['root_mean_squared_error'], 'red', label='Batch 8 Training data')
plt.plot(epochs, history_dict_5459_batch_8['val_root_mean_squared_error'], 'rx', label='Batch 8 Validation data')
plt.plot(epochs, history_5459.history['root_mean_squared_error'], 'blue', label='Batch 16 Training data')
plt.plot(epochs, history_5459.history['val_root_mean_squared_error'], 'bx', label='Batch 16 Validation data')
plt.plot(epochs, history_dict_5459_batch_32['root_mean_squared_error'], 'red', label='Batch 32 Training data')
plt.plot(epochs, history_dict_5459_batch_32['val_root_mean_squared_error'], 'rx', label='Batch 32 Validation data')
plt.plot(epochs, history_dict_5459_batch_64['root_mean_squared_error'], 'green', label='Batch 64 Training data')
plt.plot(epochs, history_dict_5459_batch_64['val_root_mean_squared_error'], 'gx', label='Batch 64 Validation data')
plt.plot(epochs, history_dict_5459_batch_128['root_mean_squared_error'], 'orange', label='Batch 128 Training data')
plt.plot(epochs, history_dict_5459_batch_128['val_root_mean_squared_error'], 'x', color = 'orange', label='Batch 128 Validation data')
plt.plot(epochs, history_dict_5459_batch_256['root_mean_squared_error'], 'black', label='Batch 256 Training data')
plt.plot(epochs, history_dict_5459_batch_256['val_root_mean_squared_error'], 'kx', label='Batch 256 Validation data')
plt.plot(epochs, history_dict_5459_batch_512['root_mean_squared_error'], 'pink', label='Batch 512 Training data')
plt.plot(epochs, history_dict_5459_batch_512['val_root_mean_squared_error'], 'x', color='pink', label='Batch 512 Validation data')
plt.plot(epochs, history_dict_5459_batch_1024['root_mean_squared_error'], 'cyan', label='Batch 1024 Training data')
plt.plot(epochs, history_dict_5459_batch_1024['val_root_mean_squared_error'], 'cx', label='Batch 1024 Validation data')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('RMSEComparation_5459.png',bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 

plt.plot(epochs, history_dict_5459_batch_8['mse'], 'red', label='Batch 8 Training data')
plt.plot(epochs, history_dict_5459_batch_8['val_mse'], 'rx', label='Batch 8 Validation data')
plt.plot(epochs, history_5459.history['mse'], 'blue', label='Batch 16 Training data')
plt.plot(epochs, history_5459.history['val_mse'], 'bx', label='Batch 16 Validation data')
plt.plot(epochs, history_dict_5459_batch_32['mse'], 'red', label='Batch 32 Training data')
plt.plot(epochs, history_dict_5459_batch_32['val_mse'], 'rx', label='Batch 32 Validation data')
plt.plot(epochs, history_dict_5459_batch_64['mse'], 'green', label='Batch 64 Training data')
plt.plot(epochs, history_dict_5459_batch_64['val_mse'], 'gx', label='Batch 64 Validation data')
plt.plot(epochs, history_dict_5459_batch_128['mse'], 'orange', label='Batch 128 Training data')
plt.plot(epochs, history_dict_5459_batch_128['val_mse'], 'x', color = 'orange', label='Batch 128 Validation data')
plt.plot(epochs, history_dict_5459_batch_256['mse'], 'black', label='Batch 256 Training data')
plt.plot(epochs, history_dict_5459_batch_256['val_mse'], 'kx', label='Batch 256 Validation data')
plt.plot(epochs, history_dict_5459_batch_512['mse'], 'pink', label='Batch 512 Training data')
plt.plot(epochs, history_dict_5459_batch_512['val_mse'], 'x', color='pink', label='Batch 512 Validation data')
plt.plot(epochs, history_dict_5459_batch_1024['mse'], 'cyan', label='Batch 1024 Training data')
plt.plot(epochs, history_dict_5459_batch_1024['val_mse'], 'cx', label='Batch 1024 Validation data')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('MSEComparation_5459.png',bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 

plt.plot(epochs, history_dict_5459_batch_8['loss'], 'red', label='Batch 8 Training data')
plt.plot(epochs, history_dict_5459_batch_8['val_loss'], 'rx', label='Batch 8 Validation data')
plt.plot(epochs, history_5459.history['loss'], 'blue', label='Batch 16 Training data')
plt.plot(epochs, history_5459.history['val_loss'], 'bx', label='Batch 16 Validation data')
plt.plot(epochs, history_dict_5459_batch_32['loss'], 'red', label='Batch 32 Training data')
plt.plot(epochs, history_dict_5459_batch_32['val_loss'], 'rx', label='Batch 32 Validation data')
plt.plot(epochs, history_dict_5459_batch_64['loss'], 'green', label='Batch 64 Training data')
plt.plot(epochs, history_dict_5459_batch_64['val_loss'], 'gx', label='Batch 64 Validation data')
plt.plot(epochs, history_dict_5459_batch_128['loss'], 'orange', label='Batch 128 Training data')
plt.plot(epochs, history_dict_5459_batch_128['val_loss'], 'x', color = 'orange', label='Batch 128 Validation data')
plt.plot(epochs, history_dict_5459_batch_256['loss'], 'black', label='Batch 256 Training data')
plt.plot(epochs, history_dict_5459_batch_256['val_loss'], 'kx', label='Batch 256 Validation data')
plt.plot(epochs, history_dict_5459_batch_512['loss'], 'pink', label='Batch 512 Training data')
plt.plot(epochs, history_dict_5459_batch_512['val_loss'], 'x', color='pink', label='Batch 512 Validation data')
plt.plot(epochs, history_dict_5459_batch_1024['loss'], 'cyan', label='Batch 1024 Training data')
plt.plot(epochs, history_dict_5459_batch_1024['val_loss'], 'cx', label='Batch 1024 Validation data')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('LossComparation_5459.png',bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(16, 12), dpi=100) 

plt.plot(epochs, history_dict_5459_batch_8['mae'], 'red', label='Batch 8 Training data')
plt.plot(epochs, history_dict_5459_batch_8['val_mae'], 'rx', label='Batch 8 Validation data')
plt.plot(epochs, history_5459.history['mae'], 'blue', label='Batch 16 Training data')
plt.plot(epochs, history_5459.history['val_mae'], 'bx', label='Batch 16 Validation data')
plt.plot(epochs, history_dict_5459_batch_32['mae'], 'red', label='Batch 32 Training data')
plt.plot(epochs, history_dict_5459_batch_32['val_mae'], 'rx', label='Batch 32 Validation data')
plt.plot(epochs, history_dict_5459_batch_64['mae'], 'green', label='Batch 64 Training data')
plt.plot(epochs, history_dict_5459_batch_64['val_mae'], 'gx', label='Batch 64 Validation data')
plt.plot(epochs, history_dict_5459_batch_128['mae'], 'orange', label='Batch 128 Training data')
plt.plot(epochs, history_dict_5459_batch_128['val_mae'], 'x', color = 'orange', label='Batch 128 Validation data')
plt.plot(epochs, history_dict_5459_batch_256['mae'], 'black', label='Batch 256 Training data')
plt.plot(epochs, history_dict_5459_batch_256['val_mae'], 'kx', label='Batch 256 Validation data')
plt.plot(epochs, history_dict_5459_batch_512['mae'], 'pink', label='Batch 512 Training data')
plt.plot(epochs, history_dict_5459_batch_512['val_mae'], 'x', color='pink', label='Batch 512 Validation data')
plt.plot(epochs, history_dict_5459_batch_1024['mae'], 'cyan', label='Batch 1024 Training data')
plt.plot(epochs, history_dict_5459_batch_1024['val_mae'], 'cx', label='Batch 1024 Validation data')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('MaeComparation_5459.png',bbox_inches='tight')

plt.show()

# Recurrent neural network con la CellId 5570

In [ ]:
simple_lstm_model_5570 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[1].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5570.summary()

simple_lstm_model_5570.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS, metrics=VAL_METRICS )

In [ ]:
history_5570 = simple_lstm_model_5570.fit(x_train_uni_cellId[1], y_train_uni_cellId[1], epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni_cellId[1], y_val_uni_cellId[1]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[1][i*5], y_val_uni_cellId[1][i*5],
                    simple_lstm_model_5570.predict(x_val_uni_cellId[1])[i*5]], 0, 'Simple LSTM model CellId 5570')
    plot.show()

In [ ]:
test_predictions_5570 = simple_lstm_model_5570.predict(x_val_uni_cellId[1]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[1], test_predictions_5570)
plt.title('Prediction accuracy CellId 5570')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_5570 - y_val_uni_cellId[1]
plt.figure(figsize=(16, 12), dpi=100) 
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
test_predictions_5570 = simple_lstm_model_5570.predict(x_val_uni_cellId[1])

mse_5570 = tf.keras.losses.MeanSquaredError()
mse_5570(y_val_uni_cellId[1], test_predictions_5570).numpy()

In [ ]:
#RMSE
rmse_5570 = tf.keras.metrics.RootMeanSquaredError()
rmse_5570(y_val_uni_cellId[1], test_predictions_5570).numpy()

# Recurrent neural network con la CellId 5644

In [ ]:
simple_lstm_model_5644 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[2].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_5644.summary()

simple_lstm_model_5644.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS, metrics=VAL_METRICS )

In [ ]:
history_5644 = simple_lstm_model_5644.fit(x_train_uni_cellId[2], y_train_uni_cellId[2], epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni_cellId[2], y_val_uni_cellId[2]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[2][i*5], y_val_uni_cellId[2][i*5],
                    simple_lstm_model_5644.predict(x_val_uni_cellId[2])[i*5]], 0, 'Simple LSTM model 5644')
    plot.show()

In [ ]:
test_predictions_5644 = simple_lstm_model_5644.predict(x_val_uni_cellId[2]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[2], test_predictions_5644)
plt.title('Prediction accuracy CellId 5644')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_5644 - y_val_uni_cellId[2]
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
test_predictions_5644 = simple_lstm_model_5644.predict(x_val_uni_cellId[2])

mse_5644 = tf.keras.losses.MeanSquaredError()
mse_5644(y_val_uni_cellId[2], test_predictions_5644).numpy()

In [ ]:
#RMSE
rmse_5644 = tf.keras.metrics.RootMeanSquaredError()
rmse_5644(y_val_uni_cellId[2], test_predictions_5644).numpy()

# Recurrent neural network con la CellId 6372

In [ ]:
simple_lstm_model_6372 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[3].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_6372.summary()

simple_lstm_model_6372.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
history_6372 = simple_lstm_model_6372.fit(x_train_uni_cellId[3], y_train_uni_cellId[3], epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni_cellId[3], y_val_uni_cellId[3]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[3][i*5], y_val_uni_cellId[3][i*5],
                    simple_lstm_model_6372.predict(x_val_uni_cellId[3])[i*5]], 0, 'Simple LSTM model CellId 6372')
    plot.show()

In [ ]:
test_predictions_6372 = simple_lstm_model_6372.predict(x_val_uni_cellId[3]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[3], test_predictions_6372)
plt.title('Prediction accuracy CellId 6372')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_6372 - y_val_uni_cellId[3]
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
test_predictions_6372 = simple_lstm_model_6372.predict(x_val_uni_cellId[3])

mse_6372 = tf.keras.losses.MeanSquaredError()
mse_6372(y_val_uni_cellId[3], test_predictions_6372).numpy()

# Recurrent neural network con la CellId 6377

In [ ]:
simple_lstm_model_6377 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[4].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_6377.summary()

simple_lstm_model_6377.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
history_6377 = simple_lstm_model_6377.fit(x_train_uni_cellId[4], y_train_uni_cellId[4], epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni_cellId[4], y_val_uni_cellId[4]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[4][i*5], y_val_uni_cellId[4][i*5],
                    simple_lstm_model_6377.predict(x_val_uni_cellId[4])[i*5]], 0, 'Simple LSTM model CellId 6377')
    plot.show()

In [ ]:
test_predictions_6377 = simple_lstm_model_6377.predict(x_val_uni_cellId[4]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[4], test_predictions_6377)
plt.title('Prediction accuracy CellId 6377')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_6377 - y_val_uni_cellId[4]
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
test_predictions_6377 = simple_lstm_model_6377.predict(x_val_uni_cellId[4])

mse_6377 = tf.keras.losses.MeanSquaredError()
mse_6377(y_val_uni_cellId[4], test_predictions_6377).numpy()

In [ ]:
#RMSE
rmse_6377 = tf.keras.metrics.RootMeanSquaredError()
rmse_6377(y_val_uni_cellId[4], test_predictions_6377).numpy()

# Recurrent neural network con la CellId 6561

In [ ]:
simple_lstm_model_6561 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni_cellId[5].shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_6561.summary()

simple_lstm_model_6561.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
history_6561 = simple_lstm_model_6561.fit(x_train_uni_cellId[5], y_train_uni_cellId[5], epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni_cellId[5], y_val_uni_cellId[5]), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni_cellId[5][i*5], y_val_uni_cellId[5][i*5],
                    simple_lstm_model_6561.predict(x_val_uni_cellId[5])[i*5]], 0, 'Simple LSTM model CellId 6561')
    plot.show()

In [ ]:
test_predictions_6561 = simple_lstm_model_6561.predict(x_val_uni_cellId[5]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[5], test_predictions_6561)
plt.title('Prediction accuracy CellId 6561')
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_6561 - y_val_uni_cellId[5]
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
#Calculo de RMSE
#rmse = tf.keras.metrics.RootMeanSquareError()
#rmse.update_state

In [ ]:
test_predictions_6561 = simple_lstm_model_6561.predict(x_val_uni_cellId[5])

mse_6561 = tf.keras.losses.MeanSquaredError()
mse_6561(y_val_uni_cellId[5], test_predictions_6561).numpy()

In [ ]:
#RMSE
rmse_6561 = tf.keras.metrics.RootMeanSquaredError()
rmse_6561(y_val_uni_cellId[5], test_predictions_6561).numpy()

# Recurrent neural network con todas las CellId

In [ ]:
BATCH_SIZE = 16
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
for x, y in val_univariate.take(20):
    plot = show_plot([x[0].numpy(), y[0].numpy()], 0, 'Simple LSTM model')
    plot.show()

In [ ]:
#from keras.preprocessing.sequence import TimeseriesGenerator
#n_input = 72
#train_univariate = tf.keras.preprocessing.sequence.TimeseriesGenerator(uni_data, uni_data,start_index = 0, end_index = TRAIN_SPLIT,
#                               length = n_input, sampling_rate=1,
#                               batch_size=1)

#val_univariate = tf.keras.preprocessing.sequence.TimeseriesGenerator(uni_data, uni_data,start_index = TRAIN_SPLIT,
#                               length = n_input, sampling_rate=1,
#                               batch_size=1)

In [ ]:
val_univariate

In [ ]:
train_univariate

In [ ]:
#model_lstm = Sequential()
#model_lstm.add(LSTM(8, input_shape=x_train_uni.shape[-2:]))
#model_lstm.add(Dense(1))
#model_lstm.summary()

#model_lstm.compile(optimizyoer='adam', loss='mae')

simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.summary()

simple_lstm_model.compile(optimizer=VAL_OPTIMIZER, loss=VAL_LOSS,metrics=VAL_METRICS)

In [ ]:
val_univariate

In [ ]:
for x, y in val_univariate.take(20):
    print(simple_lstm_model.predict(x).shape)

In [ ]:
EVALUATION_INTERVAL = 1
EPOCHS = 100

history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

In [ ]:
for x, y in val_univariate.take(20):
    plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
    plot.show()

In [ ]:
print(history.history.keys())

In [ ]:
test_predictions = simple_lstm_model.predict(x_val_uni).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni, test_predictions)
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
#x_train_uni_cellId.append(x_train_uni_frame)
#y_train_uni_cellId.append(y_train_uni_frame)
#x_val_uni_cellId.append(x_val_uni_frame)
#y_val_uni_cellId.append(y_val_uni_frame)

test_predictions_5459 = simple_lstm_model.predict(x_val_uni_cellId[0]).flatten()
test_predictions_5570 = simple_lstm_model.predict(x_val_uni_cellId[1]).flatten()
test_predictions_5644 = simple_lstm_model.predict(x_val_uni_cellId[2]).flatten()
test_predictions_6372 = simple_lstm_model.predict(x_val_uni_cellId[3]).flatten()
test_predictions_6377 = simple_lstm_model.predict(x_val_uni_cellId[4]).flatten()
test_predictions_6561 = simple_lstm_model.predict(x_val_uni_cellId[5]).flatten()
#5459	5570	5644	6372	6377	6561

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[0], test_predictions_5459, color = 'blue', label = "CellId: 5459")
plt.scatter(y_val_uni_cellId[1], test_predictions_5570, color = 'orange', label = "CellId: 5570")
plt.scatter(y_val_uni_cellId[2], test_predictions_5644, color = 'green', label = "CellId: 5644")
plt.scatter(y_val_uni_cellId[3], test_predictions_6372, color = 'red', label = "CellId: 6372")
plt.scatter(y_val_uni_cellId[4], test_predictions_6377, color = 'magenta', label = "CellId: 6377")
plt.scatter(y_val_uni_cellId[5], test_predictions_6561, color = 'brown', label = "CellId: 6561")
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions - y_val_uni
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

In [ ]:
error = test_predictions - y_val_uni
# example data
mu = 0  # mean of distribution
sigma = 0.5  # standard deviation of distribution
bins = 5
plt.hist(error, bins = bins, density=True)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("%")

In [ ]:
simple_lstm_model_acc = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model_acc.summary()

simple_lstm_model_acc.compile(optimizer='adam', loss='mae',metrics=['accuracy', 'mae', 'mse'])

In [ ]:
history_acc = simple_lstm_model_acc.fit(x_train_uni, y_train_uni, epochs=EPOCHS,
                      batch_size=16,
                      validation_data=(x_val_uni, y_val_uni), verbose=1)

In [ ]:
for i in range (0,19):
    plot = show_plot([x_val_uni[i*5], y_val_uni[i*5],
                    simple_lstm_model_acc.predict(x_val_uni)[i*5]], 0, 'Simple LSTM model')
    plot.show()

In [ ]:
test_predictions_acc = simple_lstm_model_acc.predict(x_val_uni).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni, test_predictions_acc)
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
test_predictions_acc_5459 = simple_lstm_model_acc.predict(x_val_uni_cellId[0]).flatten()
test_predictions_acc_5570 = simple_lstm_model_acc.predict(x_val_uni_cellId[1]).flatten()
test_predictions_acc_5644 = simple_lstm_model_acc.predict(x_val_uni_cellId[2]).flatten()
test_predictions_acc_6372 = simple_lstm_model_acc.predict(x_val_uni_cellId[3]).flatten()
test_predictions_acc_6377 = simple_lstm_model_acc.predict(x_val_uni_cellId[4]).flatten()
test_predictions_acc_6561 = simple_lstm_model_acc.predict(x_val_uni_cellId[5]).flatten()

plt.figure(figsize=(16, 12), dpi=100) 
a = plt.axes(aspect='equal')
plt.scatter(y_val_uni_cellId[0], test_predictions_acc_5459, color = 'blue', label = "CellId: 5459")
plt.scatter(y_val_uni_cellId[1], test_predictions_acc_5570, color = 'orange', label = "CellId: 5570")
plt.scatter(y_val_uni_cellId[2], test_predictions_acc_5644, color = 'green', label = "CellId: 5644")
plt.scatter(y_val_uni_cellId[3], test_predictions_acc_6372, color = 'red', label = "CellId: 6372")
plt.scatter(y_val_uni_cellId[4], test_predictions_acc_6377, color = 'magenta', label = "CellId: 6377")
plt.scatter(y_val_uni_cellId[5], test_predictions_acc_6561, color = 'brown', label = "CellId: 6561")
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)
xlims = [-2, 2]
ylims = [-2, 2]
plt.xlim(xlims)
plt.ylim(ylims)
_ = plt.plot(xlims, ylims)

In [ ]:
error = test_predictions_acc_5459 - y_val_uni_cellId[0]
error = np.concatenate((error, test_predictions_acc_5570 - y_val_uni_cellId[1]))
error = np.concatenate((error, test_predictions_acc_5644 - y_val_uni_cellId[2]))
error = np.concatenate((error, test_predictions_acc_6372 - y_val_uni_cellId[3]))
error = np.concatenate((error, test_predictions_acc_6377 - y_val_uni_cellId[4]))
error = np.concatenate((error, test_predictions_acc_6561 - y_val_uni_cellId[5]))
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

# Part 2: Forecast a multivariate time series

In [ ]:
features_considered = ['weekday', 'hour', 'temperature', 'internet', 'relativeHumidity']

In [ ]:
features_considered_teleco = ['callin', 'internet', 'callout', 'smsin', 'smsout']

In [ ]:
features = dataMeteoCellId[features_considered]
features.index = dataMeteoCellId['hour'] + dataMeteoCellId['weekday'] * 24 + (dataMeteoCellId['week'] - 48) * 24 * 7
features.head()

In [ ]:
features

In [ ]:
features_tempe = ['temperature', 'internet']
features_time_internet = dataMeteoCellId[features_tempe]
features_time_internet.index = dataMeteoCellId['hour'] + dataMeteoCellId['weekday'] * 24 + (dataMeteoCellId['week'] - 48) * 24 * 7

In [ ]:
features_time_internet.plot(subplots=True)

In [ ]:
features.plot(subplots=True)

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [ ]:
dataset

In [ ]:
dataset = (dataset-data_mean)/data_std

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
              labels.append(target[i+target_size])
        else:
              labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In [ ]:
past_history = 48
future_target = 6
STEP = 1

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
y_train_single

In [ ]:
x_train_single

In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [ ]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae', metrics=['mae', 'mse'])

In [ ]:
for x, y in val_data_single.take(1):
    print(single_step_model.predict(x).shape)

In [ ]:
single_step_history = single_step_model.fit(train_data_single, epochs=50,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [ ]:
def plot_train_history_mse(history, title):
    mse = history.history['mse']
    val_mse = history.history['val_mse']

    epochs = range(len(mse))

    plt.figure()

    plt.plot(epochs, mse, 'b', label='Training loss')
    plt.plot(epochs, val_mse, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('MSE')

    plt.show()

In [ ]:
plot_train_history_mse(single_step_history,
                   'Single Step Training and validation loss')

In [ ]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

# Predict a single step future

In [ ]:
for x, y in val_data_single.take(12):
    plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
    plot.show()

# Multi-Step model

In [ ]:
future_target = 6
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

In [ ]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target Internet CDR to predict : {}'.format(y_train_multi[0].shape))

In [ ]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

In [ ]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
for x, y in train_data_multi.take(1):
    multi_step_plot(x[0], y[0], np.array([0]))

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32, return_sequences=True, input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(6))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae', metrics=['mae', 'mse'])

In [ ]:
for x, y in val_data_multi.take(1):
    print (multi_step_model.predict(x).shape)

In [ ]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [ ]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

In [ ]:
for x, y in val_data_multi.take(3):
    multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim = 5, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1,activation='linear'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy','mae', 'mse'])

In [ ]:
model.summary()

In [ ]:
history1024 = model.fit(normed_X_train, y_train, epochs=1000, validation_split = 0.2, batch_size=1024, validation_data=(normed_X_test, y_test), verbose = 0, callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history1024.history)
hist['epoch'] = history1024.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
plotter.plot({'Basic': history1024}, metric = "mae")
plt.ylabel('MAE [Internet]')

In [ ]:
plotter.plot({'Basic': history1024}, metric = "mse")
plt.ylabel('MSE [Internet^2]')

In [ ]:
test_predictions = model.predict(normed_X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [Internet]')
plt.ylabel('Predictions [Internet]')
lims = [0, 8000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - y_test
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [Internet]")
_ = plt.ylabel("Count")

# Codigo para recuperarse de problemas con GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])